In [17]:
import pandas as pd
import time

start = time.time()
df = pd.read_csv('./datasets/raw/tdec.csv')

In [18]:
# df = df.sample(30, random_state=99)

In [19]:
df.columns = ['short', 'long']
df_short = df['short'] + '. ' + df['long']
df_short = [s for paragraph in df_short for s in paragraph.split('.')]
df_short = pd.DataFrame([s.strip() for s in df_short if s.strip() and len(s) > 20], columns=['short'])

In [20]:
df_long = df['short'] + '. ' + df['long']
df_long = pd.DataFrame([paragraph.strip() for paragraph in df_long if paragraph], columns=['long'])

In [21]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string

df_sw = pd.read_csv('./datasets/stopwords-id.csv')
stem = StemmerFactory().create_stemmer().stem

translation = str.maketrans('', '', string.punctuation)
def remove_punctuation(s):
    return s.translate(translation)

sw = list(df_sw['stopword'])
def remove_sw(s):
    swed = ' '.join([w for w in s.split(' ') if not w in sw])
    return swed

In [22]:
df_short['short'] = df_short['short'].apply(stem).apply(remove_punctuation)
df_short['short_no_sw'] = df_short['short'].apply(remove_sw)
df_long['long'] = df_long['long'].apply(stem).apply(remove_punctuation)
df_long['long_no_sw'] = df_long['long'].apply(remove_sw)

In [23]:
from utils import tokenize

df_short['short_no_sw_ngram'] = tokenize(df_short['short_no_sw'])
df_long['long_no_sw_ngram'] = tokenize(df_long['long_no_sw'])

In [24]:
from gensim.models.phrases import Phrases

ngram_short = Phrases(df_short['short_no_sw_ngram'], min_count=10).freeze()
df_short['short_no_sw_ngram'] = [' '.join(s) for s in ngram_short[df_short['short_no_sw_ngram']]]
ngram_long = Phrases(df_long['long_no_sw_ngram'], min_count=10).freeze()
df_long['long_no_sw_ngram'] = [' '.join(s) for s in ngram_long[df_long['long_no_sw_ngram']]]

In [28]:
df_short

,short,short_no_sw,short_no_sw_ngram
0,psi rapat ke prabowo subianto muhammad guntur ...,psi rapat prabowo subianto muhammad guntur rom...,psi rapat prabowo_subianto muhammad guntur rom...
1,muhammad guntur romli nyata keluar dari partai...,muhammad guntur romli nyata partai solidaritas...,muhammad guntur romli nyata partai_solidaritas...
2,putus itu ambil guntur telah lihat sinyal deka...,putus ambil guntur lihat sinyal partai calon p...,putus ambil guntur lihat sinyal partai calon_p...
3,prabowo belum datang kantor dpp psi pada rau 2...,prabowo kantor dpp psi rau 2 agustus 2023,prabowo kantor_dpp psi rau 2 agustus 2023
4,dalam temu itu pihak psi dan prabowo nyata mil...,temu psi prabowo nyata milik visi,temu psi prabowo nyata milik visi
...,...,...,...
177036,ikut beri uang oleh pihak lain karena lihat ya...,uang lihat uang rahmat effendi wali kota bekas...,uang lihat uang rahmat_effendi wali_kota bekas...
177037,selain itu soal wajib bayar uang ganti besar r...,wajib bayar uang ganti rp17 miliar beban rahma...,wajib_bayar uang ganti rp17 miliar beban rahma...
177038,majelis hakim adil tinggi bandung dalam putus ...,majelis hakim adil bandung putus nya berat huk...,majelis_hakim adil bandung putus nya berat huk...
177039,namun majelis hakim adil tinggi bandung belum ...,majelis hakim adil bandung sepenuh timbang tun...,majelis_hakim adil bandung sepenuh timbang tun...


In [29]:
df_long

,long,long_no_sw,long_no_sw_ngram
0,psi rapat ke prabowo subianto muhammad guntur ...,psi rapat prabowo subianto muhammad guntur rom...,psi rapat prabowo_subianto muhammad guntur rom...
1,heru budi cek sodet kali ciliwung yang baru ba...,heru budi cek sodet kali ciliwung bangun 62 pe...,heru_budi cek sodet kali_ciliwung bangun 62_pe...
2,daftar lengkap 172 obat sirop aman dari cemar ...,daftar lengkap 172 obat sirop aman cemar eg de...,daftar_lengkap 172_obat sirop aman cemar_eg de...
3,kisah mempelai meni di tengah puing bangun amb...,kisah mempelai meni puing bangun ambruk akibat...,kisah mempelai meni puing bangun ambruk akibat...
4,top metro malam tolak reuni 212 dan gulung man...,top metro malam tolak reuni 212 gulung mantra ...,top metro malam tolak reuni_212 gulung_mantra ...
...,...,...,...
7834,kpk bakal jemput paksa saksi di kasus bambang ...,kpk jemput paksa saksi bambang kayun ali fikri...,kpk jemput paksa saksi bambang_kayun ali_fikri...
7835,ridwan kamil bela bupati cianjur yang lapor ke...,ridwan kamil bela bupati cianjur lapor kpk ban...,ridwan_kamil bela bupati_cianjur lapor kpk ban...
7836,roy suryo vonis 9 bulan penjara di kasus ujar ...,roy suryo vonis 9 penjara benci meme stupa maj...,roy_suryo vonis 9 penjara benci meme_stupa maj...
7837,tanggap isu reshuffle aria bima presiden butuh...,tanggap isu reshuffle aria bima presiden butuh...,tanggap isu_reshuffle aria_bima presiden butuh...


In [30]:
# df_short.to_csv('./datasets/df_short.csv', index=False)
# df_long.to_csv('./datasets/df_long.csv', index=False)
# print(f'Elapse time: {time.time() - start}')